- - -
<!--NAVIGATION-->
Tesla Superchargers in the United States |
**[Retrieving the data](./tesla_webscraping.ipynb)** | [Cleaning the data](./tesla_clean.ipynb)
- - -

# Tesla Superchargers

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [44]:
URL = 'https://www.tesla.com/findus/list/superchargers/United+States'
main_page = requests.get(URL)


In [46]:
main_soup = BeautifulSoup(main_page.content, 'html.parser')

In [12]:
main_soup.title

<title>Tesla Superchargers in United States | Tesla</title>

In [13]:
main_soup.title.name

'title'

In [14]:
main_soup.title.string

'Tesla Superchargers in United States | Tesla'

In [15]:
main_soup.title.parent.name

'meta'

In [16]:
main_soup.a

<a href="/findus/list">Back to list</a>

#### Parse each link of supercharger locations to get the following attributes:
- station_name
- type of charger
- street address
- extended address
- city
- state
- zip
- longitude
- latitude
- number of chargers
- power capacity
- wifi
- restaurants
- lodging
- shopping
- beverage

In [47]:
all_links = []
for element in main_soup.find_all('a'):
    link = element.get('href')
    all_links.append(f'http://tesla.com{link}')

In [48]:
supercharger_links = all_links[1:]

1. Superchargers that are not yet open has a class named "coming-soon"<br><br>
2. We are only interested in those with a class named "type-charger"<br><br>
3. The attrbutes of interest are in a section element with the class name "find-us-list-details"

In [64]:
NA = 'not available'
EMPTY_STRING = ''
COMMA = ','

def getPageSoup(session, link):
    page1 = session.get(link)
    return BeautifulSoup(page1.content, 'html.parser')

def isChargingStation(soup):
    if soup.find('li', class_ = 'type-charger') == None:
        return False
    return True

def isComingSoon(soup):
    if soup.find(class_ = 'coming-soon') == None:
        return False
    return True


def findCharge(card):
    charge = card.find_all('p')
    for p in charge:
        if 'charg' in p.text.lower():
            if(p.strings):
                return p.strings
            return p.text.strip()
    return NA

def getStreetAddress(section):
    street_addr = NA
    ext_addr = NA
    line_1 = section.find('span', class_ = 'street-address')
    line_2 = section.find('span', class_ = 'extended-address')
    if(line_1):
        street_addr = line_1.text.strip()
    if(line_2):
            ext_addr = line_2.text.strip()
    return street_addr, ext_addr

def getLocality(section):
    try:
        locality = section.find('span', class_ = 'locality').string.split(COMMA)
        city = locality[0].strip()
    except AttributeError:
        return NA, NA, NA

    try:
        state_zip = locality[1].strip().split()
        state = state_zip[0]
    except IndexError:
        state_zip = locality[2].strip().split()
        state = state_zip[0]
 
    try:
        zip = state_zip[1]
    except IndexError:
        zip = NA
    return city, state, zip

def getCoordinates(section):
    coord_a = section.find('a', string = lambda text: 'driv' in text.lower())
    if not coord_a:
        return NA, NA
    coord_link = coord_a.get('href')
    if 'map' in coord_link:
        start_eq = coord_link.index('=')
        try:
            start_at = coord_link.index('@')
            start = start_eq if start_eq < start_at else start_at
        except ValueError:
            start = start_eq
        coordinates = coord_link[start +1:].split(COMMA)
        lat = coordinates[0]
        long = coordinates[1]
        return lat, long
    else:
        return NA, NA


def getChargingInfo(info_string):
    num_chargers = EMPTY_STRING
    power = EMPTY_STRING
    availability = EMPTY_STRING

    if info_string == NA:
        num_chargers = num_chargers + NA
        power = power + NA
        availability = availability + NA
    elif type(info_string) != 'str':
        lines = list(info_string)
        for line in lines:
            if 'up to' not in line:
                continue
            else:
                line = [l.strip() for l in line.split(COMMA)]
                num_chargers = num_chargers + str(line[0][0: line[0].index('S') -1]) + COMMA
                availability = availability + str(line[1]) + COMMA
                power = power + str(line[2][line[2].index('o') + 2: line[2].index('k')]) + COMMA
        num_chargers = num_chargers.strip(COMMA)
        availability = availability.strip(COMMA)
        power = power.strip(COMMA)
    else:
        try:
            info = [x.strip() for x in info_string.split(COMMA)]
            num_chargers = num_chargers + str(info[0][0: info[0].index('S') -1])
            availability = availability + str(line[1])
            power = power + str(line[2][line[2].index('o') + 2: line[2].index('k')])
        except AttributeError:
            return NA, NA, NA
    return num_chargers, availability, power


def getAmenities(section):
    try:
        amenities_icons = section.find('ul', class_ = 'amenities-icons').find_all('li')
        amens = EMPTY_STRING
        for amenity in amenities_icons:
            amens = amens + (amenity.find('a').string.strip()) + COMMA
        return amens.strip(COMMA)
    except AttributeError:
        return NA

def renderRecord(soup, URL):
    record = {
                'coming_soon': None,
                'street_address_1': None,
                'street_address_2': None,
                'city': None,
                'state': None,
                'station_name': None,
                'zip': None,
                'longitude': None,
                'latitude': None,
                'num_chargers': None,
                'availability': None,
                'power_kW': None,
                'amenities': None,
                'link': URL
            }
    if isChargingStation(soup):
        if isComingSoon(soup):
            record['coming_soon'] = True
            try:
                location = soup.find('header', class_ = 'findus-list-header').find('h1').text.split(COMMA)
                record['city'] = location[0].strip()
                record['state'] = location[1].strip()
            except:
                location = soup.find('header', class_ = 'findus-list-header').find('h1').text
                record['city'] = location
                record['state'] = NA

        else:
            record['coming_soon'] = False
            record['station_name'] = soup.find('span', class_ = 'common-name').text.strip()
            record['street_address_1'], record['street_address_2'] = getStreetAddress(soup)
            record['city'], record['state'], record['zip'] = getLocality(soup)
            vcard = soup.find('address')
            record['latitude'], record['longitude'] = getCoordinates(vcard)
            charger = findCharge(vcard)
            record['num_chargers'], record['availability'], record['power_kW'] = getChargingInfo(charger)
            record['amenities'] = getAmenities(vcard)
    return record 
            
            

In [66]:
data = []
data

[]

In [67]:
def main():
    s = requests.Session()
    for link in supercharger_links:
        soup = getPageSoup(s, link)
        data.append(renderRecord(soup, link))
    s.close()
        

In [68]:
main()

In [69]:
data_df = pd.DataFrame(data)
data_df.head()

,coming_soon,street_address_1,street_address_2,city,state,station_name,zip,longitude,latitude,num_chargers,availability,power_kW,amenities,link
0,False,FAIRFIELD INN21282 Athens-Limestone Blvd.,,Athens,AL,"Athens, AL Supercharger",35613,-86.942864,34.785416,8,available 24/7,150,"restaurants,wifi,lodging,restrooms",http://tesla.com/findus/location/supercharger/...
1,False,Tiger Crossing Shopping Center1617 South Coll...,,Auburn,AL,"Auburn, AL - South College Street Supercharger",36832,-85.498239,32.576339,12,available 24/7,250,"restaurants,shopping,beverage,restrooms",http://tesla.com/findus/location/supercharger/...
2,False,Auburn Mall1627 Opelika Road,,Auburn,AL,"Auburn, AL Supercharger",36830-2871,-85.445105,32.627837,6,available 24/7,150,"restaurants,wifi,shopping,restrooms",http://tesla.com/findus/location/supercharger/...
3,False,Uptown Entertainment District2221 Richard Arri...,,Birmingham,AL,"Birmingham, AL Supercharger",35203-1103,-86.807072,33.525826,8,available 24/7,150,"restaurants,wifi,lodging,restrooms",http://tesla.com/findus/location/supercharger/...
4,False,Holiday Inn Express & Suites Tuscaloosa 6350 ...,,Cottondale,AL,"Cottondale, AL Supercharger",35453,-87.450076,33.17466,8,available 24/7,250,"wifi,lodging,beverage,restrooms",http://tesla.com/findus/location/supercharger/...


In [70]:
data_df.to_csv('tesla_raw.csv', index=False)

- - -
<!--NAVIGATION-->
Tesla Superchargers in the United States |
**[Retrieving the data](./tesla_webscraping.ipynb)** | [Cleaning the data](./tesla_clean.ipynb)
- - -